# $$\color{red}{\text{Artificial Intelligence - Camputer Assignment 1}}$$
                                                            Search Algorithms
$$\color{lime}{\text{Alireza Javid - 810099011}}$$

## $\color{deepskyblue}{\text{Introduction}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این تمرین به کمک الگوریتم های سرچ کلاسیک که در درس فرا گرفتیم مسئله داده شده در تمرین را حل می کنیم. در ابتدا مسئله را مدلسازی می کنیم و داده های تست ها را می خوانیم. سپس با الگوریتم سرچ ناآگاهانه BFS و IDS مسئله را حل می کنیم. در انتها نیز با کمک الگوریتم *A به حل مسئله می پردازیم.
</font>
</p>

## $\color{deepskyblue}{\text{Import Libraries}}$

In [1]:
from itertools import chain
import copy
import random
import numpy as np
import heapq
from time import time

## $\color{deepskyblue}{\text{Modeling}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در ابتدا به مدلسازی مسئله می پردازیم. محیط مسئله را به صورت یک گراف در نظر می گیریم که شامل تعدادی نود می باشد. کلاس نود را به صورتی تعریف می کنیم که بتوانیم با استفاده از متد های آن به خواسته های مسئله به سادگی دست پیدا کنیم. توجه کنید داشتن همسایه های هر نود به تنهایی برای اجرای الگوریتم کافی است و نیازی به ذخیره تمامی اطلاعات نقشه نیست.
</font>
</p>

In [37]:
class Node:
    def __init__(self, id, neighbours, recipe, follower, init_leader, hard_to_pass, follower_recipe):
        self.id = id
        self.neighbours = neighbours
        self.recipe = recipe
        self.follower = follower
        self.init_leader = init_leader
        self.hard_to_pass = hard_to_pass
        self.pass_cost = 0
        self.follower_recipe = follower_recipe
        self.parent = None

    def get_neighbours(self):
        return self.neighbours

    def get_cost(self):
        if self.hard_to_pass or self.pass_cost == 0:
            self.pass_cost += 1
        return self.pass_cost

    def action(self, path):
        node_visited = path.split(' -> ')
        node_visited = map(int, node_visited)
        if self.follower:
            if all([v in node_visited for v in self.follower_recipe]):
                self.follower = False
        if self.recipe:
            self.recipe = False

    def set_parent(self, parent):
        self.parent = parent

    def __str__(self) -> str:
        path = ''
        if self.parent:
            path += str(self.parent)
            path += ' -> '
        path += str(self.id)
        return path

    def calc_path_cost(self, is_first=False):
        cost = 0
        if is_first:
            self.set_init()
        if self.parent:
            cost += self.parent.calc_path_cost()
        cost += self.get_cost()
        return cost

    def set_init(self):
        if self.parent:
            self.parent.set_init()
        self.pass_cost = 0
        
    def get_depth(self, path):
        node_visited = path.split(' -> ')
        node_visited = map(int, node_visited)
        return len(list(node_visited))

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
صف FIFO را برای پیاده سازی frontier در الگوریتم BFS به صورت زیر می سازیم.
</font>
</p>

In [3]:
class FIFO_queue:
    def __init__(self):
        self.datalist = []

    def enqueue(self, data):
        self.datalist.append(data)

    def dequeue(self):
        if self.is_empty():
            return
        data = self.datalist[0]
        self.datalist.pop(0)
        return data

    def is_empty(self):
        if len(self.datalist) == 0:
            return True
        return False

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
همچنین صف LIFO را برای پیاده سازی frontier در الگوریتم IDS به صورت زیر می سازیم.
</font>
</p>

In [4]:
class LIFO_queue:
    def __init__(self):
        self.datalist = []

    def enqueue(self, data):
        self.datalist.append(data)

    def dequeue(self):
        if self.is_empty():
            return
        data = self.datalist.pop()
        return data

    def is_empty(self):
        if len(self.datalist) == 0:
            return True
        return False

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
کلاس HEAP یا صف اولویت را به صورت زیر می سازیم تا در پیاده سازی الگوریتم *A از آن استفاده کنیم.</font>
</p>

In [5]:
class HEAP_queue:
    def __init__(self):
        self.top = 0
        self.datalist = []

    def enqueue(self, data, indicator):
        self.top += 1
        heapq.heappush(self.datalist, (indicator, self.top, data))

    def dequeue(self):
        return heapq.heappop(self.datalist)[-1]

    def is_empty(self):
        if len(self.datalist) == 0:
            return True
        return False

## $\color{deepskyblue}{\text{Importing Data}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این بخش با توجه به فرمت گفته شده در صورت مسئله داده های ورودی را می خوانیم و نقشه متناسب با مسئله را می سازیم.
</font>
</p>

In [15]:
def find_neighbours(edges, node):
    node_edges = [i for i in edges if node in i]
    neighbours = map(list, node_edges)
    neighbours = list(set(list(chain.from_iterable(neighbours))))
    neighbours.remove(node)
    return neighbours


def read_data(filename):
    with open(filename) as f:
        n, m = [int(x) for x in next(f).split()]
        edges = []
        hard_to_pass = []
        recipes = dict()
        node_neighbours = dict()
        for _ in range(m):
            edge = [int(x) for x in next(f).split()]
            edges.append(edge)
        h = int(next(f))
        hard_to_pass = [int(x) for x in next(f).split()]
        s = int(next(f))
        for _ in range(s):
            a = [int(x) for x in next(f).split()]
            recipes[a[0]] = a[2:]
        leader_init = int(next(f))
        for node in range(1, n + 1):
            node_neighbours[node] = find_neighbours(edges, node)
        return node_neighbours, recipes, hard_to_pass, n, m, leader_init


def make_map(n, recipes, node_neighbours, leader_init, hard_to_pass):
    map = dict()
    for i in range(1, n + 1):
        follower_recipe = False
        recipe = False
        follower = False
        if i in recipes.keys():
            follower = i
            follower_recipe = recipes[i]
        if i in recipes.values():
            recipe = True
        node = Node(i, node_neighbours[i], recipe, follower, leader_init == i, (i in hard_to_pass), follower_recipe)
        map[i] = node
    return map

## $\color{deepskyblue}{\text{BFS Algorithm}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
الگوریتم BFS یک الگوریتم جست و جوی ناآگاهانه می باشد. در این الگوریتم مسئله را به یک درخت تبدیل کرده و به صورت عمقی از عمق 1 شروع به جست و جو برای پیدا کردن استیت هدف می رود. در این مسئله از یک frontier که در حقیقت یک صف FIFO است برای پیاده سازی پیشگیری از جست و جوی نود تکراری استفاده می شود. همچنین استیت هر مرحله را به صورت تعداد دستور های باقی مانده و تعداد مرید های باقی مانده که دستور خود را دریافت نکرده اند و سید هنوز به آنها دستور های متناسبشان را تحویل نداده است, تعریف می کنیم. شرط پایان نیز به این صورت است که تمامی مرید ها دستور پخت های متناسب شان را دریافت کرده باشند.
</font>
</p>

In [29]:
class BFS:
    def __init__(self, node_neighbours, recipes, hard_to_pass, n, m, leader_init):
        self.map = make_map(n, recipes, node_neighbours, leader_init, hard_to_pass)
        self.leader_init = leader_init
        self.frontier = FIFO_queue()
        self.explored = []
        self.not_paied_followers = list(recipes.keys())
        self.path_cost = 0
        self.init_node = self.map[leader_init]
        self.visited_nodes = set()
        self.not_collected_recipes = set([item for sublist in list(recipes.values()) for item in sublist])
        self.all_recipes = recipes
        self.state = [self.all_recipes, self.not_paied_followers]
        self.number_of_visited = 0

    def run_bfs(self):
        self.frontier.enqueue(self.init_node)
        self.path = str(self.init_node)
        self.init_node.action(str(self.init_node))
        if self.goal_test():
            #self.show_bfs_results(self.init_node)
            self.goal = self.init_node
            return True
        while not self.frontier.is_empty():
            self.number_of_visited += 1
            self.node = self.frontier.dequeue()
            if [self.node.id, self.node_state(str(self.node))] in self.explored:
                continue
            self.explored.append([self.node.id, self.node_state(str(self.node))])
            self.visited_nodes.add(self.node.id)
            self.node_neighbours = self.node.get_neighbours()
            if self.goal_test():
                #self.show_bfs_results(self.node)
                self.goal = self.node
                return True
            for child in [self.map[x] for x in self.node_neighbours]:
                if child.id in self.visited_nodes:
                    child = copy.copy(child)
                child.set_parent(self.node)
                self.visited_nodes.add(child.id)
                child.action(str(child))
                if [child.id, self.node_state(str(child))] not in self.explored:
                    if self.goal_test():
                        self.path_cost += child.get_cost()
                        #self.show_bfs_results(child)
                        self.goal = child
                        return True
                    self.frontier.enqueue(child)

    def show_bfs_results(self, node=None):
        if not node:
            node = self.goal
        self.path_cost = node.calc_path_cost()
        print("Answer is : ", self.path_cost - 1)
        print("# visited nodes : ", self.number_of_visited)
        print(str(node))

    def goal_test(self):
        if self.state[1]:
            return False
        else:
            return True

    def node_state(self, path):
        node_visited = path.split(' -> ')
        node_visited = map(int, node_visited)
        recipes = copy.copy(self.not_collected_recipes)
        followers = copy.copy(self.not_paied_followers)
        for node in node_visited:
            if node in recipes:
                recipes.remove(node)
            if node in followers:
                if sum([v in self.all_recipes[node] for v in recipes]) < 1:
                    followers.remove(node)
        self.state = [recipes, followers]
        return self.state

## $\color{deepskyblue}{\text{IDS Algorithm}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
الگوریتم IDS یک الگوریتم جست و جوی ناآگاهانه دیگر است که در قسمت زیر آن را پیاده کرده ایم. در این الگوریتم, الگوریتم DFS به صورت مرحله ای و با عمق محدود اجرا می شود و در هر مرحله عمق آن زیاد می شود. تعریف هر استیت در این بخش نیز مانند بخش های قبل است. این الگوریتم زمان بسیار زیادی را نسبت به الگوریتم قبلی برای یافتن جواب نیاز دارد. این الگوریتم برای پیاده سازی frontier از یک صف LIFO  استفاده می کند(مانند الگوریتم DFS). تعریف استیت و شرط پایانی همانند الگوریتم BFS می باشد.</font>
</p>

In [39]:
class IDS:
    def __init__(self, node_neighbours, recipes, hard_to_pass, n, m, leader_init):
        self.map = make_map(n, recipes, node_neighbours, leader_init, hard_to_pass)
        self.leader_init = leader_init
        self.frontier = LIFO_queue()
        self.explored = []
        self.not_paied_followers = list(recipes.keys())
        self.path_cost = 0
        self.init_node = self.map[leader_init]
        self.visited_nodes = set()
        self.not_collected_recipes = set([item for sublist in list(recipes.values()) for item in sublist])
        self.all_recipes = recipes
        self.state = [self.all_recipes, self.not_paied_followers]
        self.number_of_visited = 0

    def run_ids(self):
        result = None
        depth = 0
        self.explore_depth = dict()
        while result is None:
            result = self.dls(self.map[self.leader_init], depth)
            depth += 1
        #self.show_ids_results(result)
        self.goal = result

    def dls(self, init_node, depth):
        self.frontier.enqueue(self.init_node)
        self.init_node.action(str(self.init_node))
        if self.goal_test():
            return init_node
        while not self.frontier.is_empty():
            self.number_of_visited += 1
            self.node = self.frontier.dequeue()
            if [self.node.id, self.node_state(str(self.node))] not in self.explored:
                self.explored.append([str(self.node), self.node_state(str(self.node))])
                self.visited_nodes.add(self.node.id)
                self.explore_depth[str(self.node)] = depth
            if self.node.get_depth(str(self.node)) < depth:
                node_neighbours = self.node.get_neighbours()
                for child in [self.map[x] for x in node_neighbours]:
                    if child.id in self.visited_nodes:
                        child = copy.copy(child)
                    child.set_parent(self.node)
                    self.visited_nodes.add(child.id)
                    if not self.is_dup(str(child)):
                        continue
                    state = [str(child), self.node_state(str(child))]
                    if state not in self.explored:
                        child.action(str(child))
                        if self.goal_test():
                            return child
                    if (state not in self.explored) or \
                            depth > self.explore_depth[str(child)]:
                        self.frontier.enqueue(child)

    def goal_test(self):
        if self.state[1]:
            return False
        else:
            return True

    def node_state(self, path):
        node_visited = path.split(' -> ')
        node_visited = map(int, node_visited)
        recipes = copy.copy(self.not_collected_recipes)
        followers = copy.copy(self.not_paied_followers)
        for node in node_visited:
            if node in recipes:
                recipes.remove(node)
            if node in followers:
                if sum([v in self.all_recipes[node] for v in recipes]) < 1:
                    followers.remove(node)
        self.state = [recipes, followers]
        return self.state

    def is_dup(self, path):
        node_visited = path.split(' -> ')
        node_visited = list(map(int, node_visited))
        recipes = self.state[0]
        followers = self.state[1]
        node_list = []
        i = -1
        for node in node_visited:
            i += 1
            if node not in node_list:
                node_list.append(node)
            else:
                index = node_list.index(node)
                dup_list = list(node_visited[index + 1:i])
                if sum([v in recipes for v in dup_list]) > 0:
                    return True
                elif sum([v in followers for v in dup_list]) > 0:
                    for second_node in dup_list:
                        if second_node in self.all_recipes.keys() and \
                                sum([v in self.all_recipes[second_node] for v in recipes]) < 1:
                            return True
                else:
                    return False
        return True
    def show_ids_results(self, node=None):
        if not node:
            node = self.goal
        self.path_cost = node.calc_path_cost()
        print("Answer is : ", self.path_cost - 1)
        print("# visited nodes : ", self.number_of_visited)
        print(str(node))

## $\color{deepskyblue}{\text{A* Algorithm}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
الگوریتم *A یک الگوریتم سرچ آگاهانه است که در بخش زیر آن را پیاده سازی کردیم. مبنای این الگوریتم بر اساس تابع f(n) می باشد که از طرق رابطه زیر تعریف می شود.
</font>
</p>

$$
f(n) = \alpha h(n) + g(n)
$$

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در اینجا g(n) مسیر طی شده تا حال حاضر است و h(n) هزینه heuristic و هزینه تخمینی می باشد که در ادامه برای این مسئله تعریف می کنیم. در پیاده سازی این الگوریتم از یک صف اولویت متناسب با مقدار f(n) استفاده می شود. همچنین مقدار آلفا در الگوریتم *A ساده برابر 1 می باشد.
</font> 
</p>

### $\color{deepskyblue}{\text{Heuristic}}$
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
مسئله هنگامی حل خواهد شد که تمامی دستور ها به مرید ها تحویل داده شود. ما نیز از همین موضوع استفاده می کنیم و هیروستیک نود را با توجه به مسیر طی شده تا آن و تعداد دستور های برداشته شده و یا تجویل داده شده در نظر می گیریم. یعنی منفی اندازه نود هایی که در مسیرشان دستور ها برداشته شده و یا به مرید ها تحویل داده می شود را حساب می کنیم. این هیروستیک consistent نیز می باشد زیرا داریم: 
</font> 
</p>

$$
h(n) \le h^*(n)
$$
$$
cost(AtoC) \ge h(A)−h(C)
$$

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
یعنی برای رسیدن به مقصد باید هزینه برداشتن دستور ها و تحویل ها را باز هم طی کنیم.
</font> 
</p>

In [44]:
class Astar:
    def __init__(self, node_neighbours, recipes, hard_to_pass, n, m, leader_init, alpha):
        self.map = make_map(n, recipes, node_neighbours, leader_init, hard_to_pass)
        self.leader_init = leader_init
        self.frontier = HEAP_queue()
        self.explored = []
        self.not_paied_followers = list(recipes.keys())
        self.path_cost = 0
        self.init_node = self.map[leader_init]
        self.visited_nodes = set()
        self.not_collected_recipes = set([item for sublist in list(recipes.values()) for item in sublist])
        self.all_recipes = recipes
        self.state = [self.all_recipes, self.not_paied_followers]
        self.alpha = alpha
        self.number_of_visited = 0

    def h(self, path):
        node_visited = path.split(' -> ')
        node_visited = map(int, node_visited)
        recipes = copy.copy(self.not_collected_recipes)
        followers = copy.copy(self.not_paied_followers)
        h_cost = 0
        for node in node_visited:
            if node in recipes:
                h_cost -= 1
            if node in followers:
                if sum([v in self.all_recipes[node] for v in recipes]) < 1:
                    h_cost -= 1
        return h_cost

    def g(self, node):
        return node.calc_path_cost(True)

    def f(self, node):
        return self.alpha * self.h(str(node)) + self.g(node)

    def run_Astar(self):
        self.frontier.enqueue(self.init_node, self.f(self.init_node))
        self.path = str(self.init_node)
        self.init_node.action(str(self.init_node))
        if self.goal_test():
            #self.show_astar_results(self.init_node)
            self.goal = self.init_node
            return True
        while not self.frontier.is_empty():
            self.number_of_visited += 1
            self.node = self.frontier.dequeue()
            if [self.node.id, self.node_state(str(self.node))] in self.explored:
                continue
            self.explored.append([self.node.id, self.node_state(str(self.node))])
            self.visited_nodes.add(self.node.id)

            self.node_neighbours = self.node.get_neighbours()
            if self.goal_test():
                #self.show_astar_results(self.node)
                self.goal = self.node
                return True
            for child in [self.map[x] for x in self.node_neighbours]:
                if child.id in self.visited_nodes:
                    child = copy.copy(child)
                child.set_parent(self.node)
                self.visited_nodes.add(child.id)
                child.action(str(child))
                if [child.id, self.node_state(str(child))] not in self.explored:
                    if self.goal_test():
                        self.path_cost += child.get_cost()
                        #self.show_astar_results(child)
                        self.goal = child
                        return True
                    self.frontier.enqueue(child, self.f(child))

    def show_astar_results(self, node=None):
        if not node:
            node = self.goal
        self.path_cost = node.calc_path_cost()
        print("Answer is : ", self.path_cost - 1)
        print("# visited nodes : ", self.number_of_visited)
        print(str(node))

    def goal_test(self):
        if self.state[1]:
            return False
        else:
            return True

    def node_state(self, path):
        node_visited = path.split(' -> ')
        node_visited = map(int, node_visited)
        recipes = copy.copy(self.not_collected_recipes)
        followers = copy.copy(self.not_paied_followers)
        for node in node_visited:
            if node in recipes:
                recipes.remove(node)
            if node in followers:
                if sum([v in self.all_recipes[node] for v in recipes]) < 1:
                    followers.remove(node)
        self.state = [recipes, followers]
        return self.state

## $\color{deepskyblue}{\text{Running BFS}}$

In [25]:
def run_BFS(test_file):
	node_neighbours, recipes, hard_to_pass, n, m, leader_init = read_data(test_file)
	average_time = 0
	for i in range(0, 3):
		bfs = BFS(node_neighbours, recipes, hard_to_pass, n, m, leader_init)
		start_time = time()
		bfs.run_bfs()
		finish_time = time()
		average_time += (finish_time - start_time)
	bfs.show_bfs_results()
	average_time /= 3 
	print("Average Execution Time for 3 BFS runs: " + str(average_time) + " seconds\n")

In [30]:
print("BFS Testcases\n")
files = ["input.txt", "input2.txt", "input3.txt"]
for test in files:
    i = files.index(test) + 1
    print("Testcase #",i)
    run_BFS(test)

BFS Testcases

Testcase # 1
Answer is :  8
# visited nodes :  41
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Average Execution Time for 3 BFS runs: 0.0016620953877766926 seconds

Testcase # 2
Answer is :  12
# visited nodes :  9710
28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 22 -> 28
Average Execution Time for 3 BFS runs: 2.0910750230153403 seconds

Testcase # 3
Answer is :  21
# visited nodes :  5452
40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Average Execution Time for 3 BFS runs: 1.243674596150716 seconds



## $\color{deepskyblue}{\text{Running IDS}}$

In [41]:
def run_IDS(test_file):
	node_neighbours, recipes, hard_to_pass, n, m, leader_init = read_data(test_file)
	average_time = 0
	for i in range(0, 3):
		ids = IDS(node_neighbours, recipes, hard_to_pass, n, m, leader_init)
		start_time = time()
		ids.run_ids()
		finish_time = time()
		average_time += (finish_time - start_time)
	ids.show_ids_results()
	average_time /= 3 
	print("Average Execution Time for 3 IDS runs: " + str(average_time) + " seconds\n")

In [49]:
print("IDS Testcases\n")
files = ["input.txt", "input2e.txt", "input3e.txt"]
for test in files:
    i = files.index(test) + 1
    print("Testcase #",i)
    run_IDS(test)

IDS Testcases

Testcase # 1
Answer is :  8
# visited nodes :  588
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Average Execution Time for 3 IDS runs: 0.08776521682739258 seconds

Testcase # 2
Answer is :  8
# visited nodes :  850
9 -> 10 -> 6 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Average Execution Time for 3 IDS runs: 0.12134210268656413 seconds

Testcase # 3
Answer is :  13
# visited nodes :  36783
13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Average Execution Time for 3 IDS runs: 62.18430685997009 seconds



## $\color{deepskyblue}{\text{Running A*}}$

In [51]:
def run_Astar(test_file, alpha):
	node_neighbours, recipes, hard_to_pass, n, m, leader_init = read_data(test_file)
	average_time = 0
	for i in range(0, 3):
		astar = Astar(node_neighbours, recipes, hard_to_pass, n, m, leader_init, alpha)
		start_time = time()
		astar.run_Astar()
		finish_time = time()
		average_time += (finish_time - start_time)
	astar.show_astar_results()
	average_time /= 3 
	print("Average Execution Time for 3 A* runs with alpha = " + str(alpha) + ": "+ str(average_time) + " seconds\n")

In [48]:
print("A* Testcases\n")
files = ["input.txt", "input2.txt", "input3.txt"]
for test in files:
    i = files.index(test) + 1
    print("Testcase #",i)
    run_Astar(test, 1)

A* Testcases

Testcase # 1
Answer is :  9
# visited nodes :  40
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Average Execution Time for 3 A* runs with alpha = 1: 0.003324270248413086 seconds

Testcase # 2
Answer is :  12
# visited nodes :  5180
28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 22 -> 28
Average Execution Time for 3 A* runs with alpha = 1: 1.1027181148529053 seconds

Testcase # 3
Answer is :  22
# visited nodes :  5154
40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Average Execution Time for 3 A* runs with alpha = 1: 1.3646843433380127 seconds



## $\color{deepskyblue}{\text{Weighted A*}}$

In [53]:
print("A* with alpha = 3 Testcases\n")
files = ["input.txt", "input2.txt", "input3.txt"]
alpha = 3
for test in files:
    i = files.index(test) + 1
    print("Testcase #",i)
    run_Astar(test, alpha)

A* Testcases

Testcase # 1
Answer is :  9
# visited nodes :  20
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Average Execution Time for 3 A* runs with alpha = 3: 0.001994053522745768 seconds

Testcase # 2
Answer is :  40
# visited nodes :  106
28 -> 19 -> 3 -> 11 -> 3 -> 13 -> 3 -> 11 -> 24 -> 9 -> 24 -> 11 -> 3 -> 13 -> 23 -> 5 -> 2 -> 9 -> 24 -> 11 -> 3 -> 13 -> 1 -> 7 -> 11 -> 3 -> 13 -> 6 -> 9 -> 24 -> 17 -> 29 -> 4 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 22 -> 28
Average Execution Time for 3 A* runs with alpha = 3: 0.04788311322530111 seconds

Testcase # 3
Answer is :  65
# visited nodes :  1279
40 -> 42 -> 38 -> 24 -> 31 -> 24 -> 38 -> 42 -> 27 -> 47 -> 27 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 30 -> 45 -> 31 -> 24 -> 38 -> 42 -> 27 -> 47 -> 49 -> 9 -> 49 -> 47 -> 27 -> 42 -> 38 -> 24 -> 31 -> 45 -> 12 -> 16 -> 12 -> 45 -> 31 -> 24 -> 38 -> 42 -> 27 -> 47 -> 49 -> 43 -> 19 -> 43 -> 49 -> 47 -> 27 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 10 -> 50 -> 25
Average Execution 

In [54]:
print("A* with alpha = 5 Testcases\n")
files = ["input.txt", "input2.txt", "input3.txt"]
alpha = 5
for test in files:
    i = files.index(test) + 1
    print("Testcase #",i)
    run_Astar(test, alpha)

A* with alpha = 5 Testcases

Testcase # 1
Answer is :  9
# visited nodes :  16
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Average Execution Time for 3 A* runs with alpha = 5: 0.0013306140899658203 seconds

Testcase # 2
Answer is :  40
# visited nodes :  96
28 -> 19 -> 3 -> 11 -> 3 -> 13 -> 3 -> 11 -> 24 -> 9 -> 24 -> 11 -> 3 -> 13 -> 23 -> 5 -> 2 -> 9 -> 24 -> 11 -> 3 -> 13 -> 1 -> 7 -> 11 -> 3 -> 13 -> 6 -> 9 -> 24 -> 17 -> 29 -> 4 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 22 -> 28
Average Execution Time for 3 A* runs with alpha = 5: 0.06017168362935384 seconds

Testcase # 3
Answer is :  65
# visited nodes :  239
40 -> 42 -> 38 -> 24 -> 31 -> 24 -> 38 -> 42 -> 27 -> 47 -> 27 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 30 -> 45 -> 31 -> 24 -> 38 -> 42 -> 27 -> 47 -> 49 -> 9 -> 49 -> 47 -> 27 -> 42 -> 38 -> 24 -> 31 -> 45 -> 12 -> 16 -> 12 -> 45 -> 31 -> 24 -> 38 -> 42 -> 27 -> 47 -> 49 -> 43 -> 19 -> 43 -> 49 -> 47 -> 27 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 10 -> 50 -> 25
Aver

## $\color{deepskyblue}{\text{Compare algorithms with each other}}$

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<ul dir =rtl>
<ul>
  <li>BFS</li>
  این الگوریتم ناآگاهانه تقریبا سرعت خوبی دارد و با توجه به آن که نود هایی با استیت تکراری را دوباره پیمایش نمی کند می تواند با سرعت معقولی به جواب نهایی برسد. 
    <ul>
      <li>
        <p style="color:green;">مزیت ها</p>
      </li>
      <ol>
      <li>در صورتی که مسئله پاسخ داشته باشد الگوریتم BFS آن را پیدا میکند. </li>
      <li> .این الگوریتم در صورت وجود چند جواب, بهترین جواب یا پاسخ بهینه را بر میگرداند</li>
      <li>این الگوریتم در لوپ یا نود های بی انتها گیر نخواهد کرد. </li>
      </ol>
      <li>
        <p style="color:red;">معایب</p>
      </li>
      <ol>
      <li>نیاز به حافظه بالایی دارد.</li>
      <li>ممکن است زمان زیادی برای یافتن جواب نیاز داشته باشد.</li>
      </ol>
</ul> 
  <li>IDS</li>
  این الگوریتم ناآگاهانه حداکثر عمق الگوریتم DFS را مرحله ای زیاد می کند و اجرای آن به شدت کند می باشد اما مصرف حافظه آن کم است. 
    <ul>
      <li>
        <p style="color:green;">مزیت ها</p>
      </li>
      <ol>
      <li>مصرف حافظه این الگوریتم کمتر از BFS است.</li>
      <li> .این الگوریتم در صورت وجود چند جواب, بهترین جواب یا پاسخ بهینه را بر میگرداند</li>
      </ol>
      <li>
        <p style="color:red;">معایب</p>
      </li>
      <ol>
      <li>محاسبات تکراری در اجرای این الگوریتم بسیار زیاد است و سرعت اجرای آن به شدت کند می باشد.</li>
      </ol>
      </ul> 
  <li>*A</li>
  این الگوریتم آگاهانه به کمک تابع h(n) مناسب می تواند با سرعت خوبی پاسخ بهینه را پیدا کند. تنها باید این تابع هیروستیک شرط consistency را داشته باشد.
      <ul>
      <li>
        <p style="color:green;">مزیت ها</p>
      </li>
      <ol>
      <li>آگاهانه و سریع است.</li>
      <li> .این الگوریتم در صورت وجود چند جواب, بهترین جواب یا پاسخ بهینه را بر میگرداند</li>
      <li>مصرف حافظه بالایی ندارد.</li>
      </ol>
      <li>
        <p style="color:red;">معایب</p>
      </li>
      <ol>
      <li>یافتن هیروستیک مناسب می تواند زمان بر باشد.</li>
      </ol>
      </ul> 
  <li>*Weighted A</li>
این الگوریتم آگاهانه بسیار شبیه به *A است اما سریع تر است و برای تابع h(n) یک ضریب در نظر می گیرد که سرعت رسیدن به پاسخ را افزایش می دهد.
    <ul>
      <li>
        <p style="color:green;">مزیت ها</p>
      </li>
      <ol>
      <li>سرعت اجرا و یافتن پاسخ مسئله بسیار بالا است و اختلاف چشمگیری با راه های دیگر دارد.</li>
      </ol>
      <li>
        <p style="color:red;">معایب</p>
      </li>
      <ol>
      <li>در صورت وجود چند پاسخ ممکن است پاسخ غیر بهینه را بازگرداند.</li>
      </ol>
</ul>
</ul> 

</font> 
</p>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
می توان نتیجه گرفت که الگوریتم BFS برای شرایطی مناسب است که:
<ul dir =rtl>
 <ul>
  <li>محدودیت حافظه و حافظه نداریم</li>
  <li>پیدا کردن هیروستیک مناسب دشوار است.</li>
</ul> 
</font> 
</p>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
همچنین الگوریتم A* برای شرایطی مناسب است که:
<ul dir =rtl>
 <ul>
  <li>هیروستیک مناسب داریم</li>
  <li>محدودیت زمانی داریم.</li>
</ul> 



همچنین به صورت کلی الگوریتم های جست و جوی آگاهانه بسیار سریع تر و مناسب تر از الگوریتم های ناآگاهانه می باشد.
</font> 
</p>
